In [1]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import pandas as pd
import numpy as np
import blackboard
import knowledge_agent as ka
import time
import train_surrogate_models as tm
import random

# Building a Blackboard System

We start by initializing our multi-agent system using osBrain [1].
For the basic program, three agents will be required.

* Agent 1: Blackboard
* Agent 2: Knowledge Agent - Neutronics
* Agent 3: Knowledge Agent - Blackboard Level 2 Synthesizer

The blackboard agent will retain all of the optimization results obtained by the neutronics agent on the third level of the blackboard.
Abstract level 3 contains raw data in the form of Pandas Dataframe. 
This data consists of the design variables (height, smear, and plutonium fraction), objectives (keff, void coefficient, doppler coefficient, plutonium fraction), and the weights for each objective.

The neutronics knowledge agent (KA) will run a neutronics optimization for a given set of objective weights.
For the traditional blackboard system, as seen here, we use a proxy neutronics KA, which as features uniques to performing sodium fast reactor optimization.
This is performed using Dakota as the optimization engine, and a surrogate model to obtain the reactor paramters (i.e. our objective functions).
Once an optimal solution has been determined, the neutronics KA writes the associated design variables, objective functions, and weights to the blackboard.

The Blackboard Level 2 Synthesizer KA examines abstract level 3 of the blackboard to determine if solutions are close to the desired solution.
A proxy is also used for the Blackboard Level 2 Synthesizer, as the propertires associated with core design are slightly different compared with the full scale MABS.
If a solution is within some percent of the desired solution, the Blackboard Level 2 Synthesizer KA will take basic information from abstract level 3 and place it in abstract level 2.

## Agent Initialization

We start off by simply initializing the blackboard and neutronics KA.
Both agents are initialized using the `connect_agent` function.
This connects each agent to the blackboard and connects the three different types of communication that are required to interact with the blackboard.
The blackboard is the only agent that knowledge agents interact with.
KA never talk with other KA, and only pass information to each other by writing to the blackboard. 
This ia a fundamental property of blackboard systems, even in a multi-agent environment.
Three channels of communicationa are used in the blackboard system: writer, triger, and excecute.

The writer channel is a request-reply communication channel, which allows the KA to continually request if the blackboard is currently being written to.
If it is, the blackboard informs the KA that another KA is writing and that it will have to wait.
The KA will then wait for a second and request the state of the blackboard again, until it is able to write its information.
For the traditional blackboard, only one knowledge agent is active at a time, and as such it should never encounter a situation where it will be denied access to the blackboard.

The trigger channel contains two communication channels, a publish-subscribe channel and a push-pull channel.
The blackboard initiates the publish-subscribe channel which allows the blackboard publish to trigger event.
The trigger event is used to determine which KA will be selected for execution in the next step.
The KAs initiate the push-pull channel, which allows them respond to the trigger publication with their associated trigger value.
Trigger values will change for some KAs depending on how the problem is progressing.
For the traditional blackboard scheme, the trigger value for the neutroncis knowledge agent will always be 1.0, and the blackboard level 2 synthesizer's trigger value will oscilate between 0 and 2 every 10 evaluations to periodically check the blackboard for a good solution.

The execute channel is a push-pull channel, where the blackboard will inform a KA that they have been selected for execution.
We then enter a while loop until the KA has finished their action for the traditional blackboard system.
Otherwise, the blackboard could continue on with the process and send another tirgger event.

In [2]:
ns = run_nameserver()
bb = run_agent(name='blackboard', base=blackboard.Blackboard)
ka_rp = run_agent(name='ka_rp', base=ka.KaReactorPhysics_Proxy)
#ka_rp1 = run_agent(name='ka_rp1', base=ka.KaReactorPhysics_Proxy)
ka_lvl2 = run_agent(name='ka_lvl2', base=ka.KaBbLvl2_Proxy)

def connect_agent(agent, bb):
    agent.add_blackboard(bb)
    agent.connect_writer()
    agent.connect_trigger()
    agent.connect_execute()
    if 'rp' in agent.get_attr('name'):
        agent.set_attr(objectives=['keff', 'void_coeff', 'doppler_coeff', 'pu_content'])
        agent.set_attr(design_variables=['height', 'smear', 'pu_content'])
        agent.set_attr(function_evals=10)
        agent.set_attr(results_path='/Users/ryanstewart/projects/Dakota_Interface/GA/mabs_results/')
    elif 'lvl2' in agent.get_attr('name'):
        agent.set_attr(ind_err=0.1)

connect_agent(ka_rp, bb)
#connect_agent(ka_rp1, bb)
connect_agent(ka_lvl2, bb)
#bb.set_attr(_DEBUG=True)
#ka_lvl2.set_attr(_DEBUG=True)
#ka_rp.set_attr(_DEBUG=True)

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:19629 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:19629
INFO [2020-03-02 13:22:33.715334] (blackboard): BB connected writer to ka_rp
INFO [2020-03-02 13:22:33.737052] (blackboard): BB connected writer to ka_lvl2
INFO [2020-03-02 13:22:37.224389] (blackboard): Determining which KA to execute
INFO [2020-03-02 13:22:38.230369] (blackboard): Selecting agent ka_rp (trigger value: 1.0) to execute
INFO [2020-03-02 13:22:38.233209] (ka_rp): Executing agent ka_rp
INFO [2020-03-02 13:22:42.060804] (blackboard): Agent ka_rp given permission to write
INFO [2020-03-02 13:22:43.250031] (blackboard): Determining which KA to execute
INFO [2020-03-02 13:22:44.255047] (blackboard): Selecting agent ka_rp (trigger value: 1.0) to execute
INFO [2020-03-02 13:22:44.257170] (ka_rp): Executing agent ka_rp
INFO [2020-03-02 13:22:48.084674] (blackboard): Agent ka_rp given permission to write
INFO [2020-03-02 13:22:49.275748] (blackboard): Dete

# Running the Blackboard System

The multi-agent module used for the blackboard system is osBrain.
osBrain requires that the blackboard sytem be run externally from the blackboard agent.
If the loop seen below is implemented into a function within the blackboard, we get errors associated with blackboard communication.
This is due to the fact that each agent is a separate process, and when the blackbaord runs a method, it consumes the entire process, prevent the blackboard from processing incoming messages [1].
Future iterations of the blackboard system may split the blackbaord into a controller and blackboard segment to seperate the communication and administration portions.

Below we see a simple loop which iterates over the three major steps associated with the blackboard: trigger, controller, and execute).
As noted previously, the trigger publishes a request for trigger values, when the agents have responsed the controller is initiated, a waiting period of 1.0 seconds is used to ensure the trigger communication has enough time to process the responses.
The controller examines all of the KA's trigger values and selects the knowledge agent with the highest trigger values.
Execute sends a messge the the KA with the highest trigger value and tell is to execute its main code.

This will allow us to see how the Dakota interface operates, and will run 10 Dakota optimizations for 10 different weighting schemes. 
10 optimization will allow for the Blackboard Level 2 Synthesizer to examine the results and determine if any solution is within 10% of our desired solution.
Note: due to the random nature in assigning the objective weights you will not always get a solution present in blackboard abstract level 2 (`lvl_2`).

In [ ]:
def wait_for_ka(bb):
    if len(bb.get_attr('lvl_3').keys()) > 20:
        bb.build_surrogate_models_propxy()
    while not bb.get_attr('new_entry'):
        time.sleep(1)
    bb.set_attr(new_entry=False)
    
for i in range(20):
    bb.publish_trigger()
    time.sleep(1)
    bb.controller()
    time.sleep(1)
    bb.send_execute()
    time.sleep(1)
    wait_for_ka(bb)
    print('Finished trigger event: {}'.format(bb.get_attr('trigger_event_num')))
    
ns.shutdown()

Finished trigger event: 1
Finished trigger event: 2
Finished trigger event: 3
Finished trigger event: 4
Finished trigger event: 5
Finished trigger event: 6
Finished trigger event: 7
Finished trigger event: 8
Finished trigger event: 9
Finished trigger event: 10
Finished trigger event: 11
Finished trigger event: 12


# Behind the Scenes

The previous two aspects have shown off the blackboard and its interactions with the two types of knowledge agents, but from the outside, the knowledge agents act like a blackbox; we tell the agent to run and it returns some value to the blackboard.
The glean a better understanding of the blackboard system as a whole, an more in-depth discussion of the knowledge agents inner workings is required.

## Neutronics (Reactor Physics) Agent

The neutronics agent (`ka_rp`) initiates Dakota to perform a single-objective optimization scheme using a gentic algorithm (SOGA) [2].
For each SOGA, Dakota attempts to create an optimized core by adjusting the design variables: fuel height, smear, and plutonium fraction.
An optimized core is dependent on the three objective functions: k-eigenvalue, sodium void coefficent, and Doppler coefficient.

To determine the set of objectives based on the design variables, a surrogate model is used rather the perform a full MCNP calculation [3].
The surrogate model uses a database to perform an interpolation between the known values present in the database, and the values Dakota requires.
The module `train_surrogate_models` is used as the surrogate model.

Upon completion of an optimization scheme, the `ka_rp` writes the optimal core configuration to the blackboard on the third level of abstraction (`lvl_3`), where raw reactor physics data is kept.
This includes writing the design variables, objective function values, and objective function weights.

## Blackboard Level 2 Synthesizer Agent

The blackboard level 2 synthesizer agent (`ka_lvl2`) reads the blackboard and examines each of the solutions to determine if any solution is within some percent of the desired solution.
The desired solutions is based on utilizing physical programming in place of a weighting scheme for the SOGA [4]. 
The solutions should have the following objective function values:

* k-eigenvalue: 1.0303
* Void Coefficient: -110.023
* Doppler Coefficient: -0.6926
* Pu Fraction: 54.75

The `ka_lvl2` agent find the percent difference between each objective and sums these values. 
If the resulting sum is less than the `ind_err` set for `ka_lvl2` then it gets placed in abstract level 2 of the blackbaord (`lvl_2`).

## Blackboard Agent

The blackboard is unique agent which holds the all of the information obtained by the `ka_rp` agent, and information that has been updated by `ka_lvl2`.
This information is stored in two separate abstract levels as hinted at in the previous sections.

Abstract level 3 (`lvl_3`) contains data from the `ka_rp` in the form of a dictonary, whose keys are the core name (in the form of `Core_(weights)`).
The value for each dictionary is a nested dictionary with the keys for the reactor paramters (`rx_parameters`) and the cross-section set used (`xs_set`).
The cross-section set is currently not used for this analysis, but will be used in future research.
The `rx_parameters` entry contains a pandas dataframe consisting of various reactor data information including all of the design and objective variables along with the weights associated with each objective variable.

Abstract level 2 (`lvl_2`) stores core designs that meet the error requirements set up by `ka_lvl2`.


# References

[1] osBrain v0.6.5, (2019), GitHub repository, https://github.com/opensistemas-hub/osbrain.

[2] B.M. Adams, et al, “Dakota, A Multilevel Parallel Object-Oriented Framework for DesignOptimization, Parameter Estimation, Uncertainty Quantification, and Sensitivity Analysis:Version 6.10 User’s Manual,” Sandia Technical Report SAND2014-4253, May 2019.

[3] C.J. Werner,  et al.,  “MCNP6.2 Release Notes”,  Los Alamos National Laboratory,  reportLA-UR-18-20808 (2018).

[4] R. Stewart and T.S. Palmer, "Utilizing a Reduced-Order Model and Physical Programming for Preliminary Reactor Design Optimization," PHYSOR-2020, Cambridge, UK, 2020.

# Data Extraction 

The previous cell built and ran 5 different Dakota optimization runs to examine the effects that the weighting scheme has on the objectives.
The neutronics KA obtained this informatin using a surrogate model to perform the optimization proces.
Results were then written to the blackboard, where they can easily be examined.
Below we print out the optimized design variables along with the associated objective functions.


In [ ]:
lvl_3 = bb.get_attr('lvl_3')
for k,v in lvl_3.items():
    print('Reactor: {} \n Height: {} Smear: {} Pu: {} \n keff: {} Void: {}  Doppler: {} \n Weights : ({},{},{},{})'.format(k, 
        v['reactor_parameters']['height'][k], v['reactor_parameters']['smear'][k], v['reactor_parameters']['pu_content'][k], round(v['reactor_parameters']['keff'][k],4), round(v['reactor_parameters']['void'][k],2), round(v['reactor_parameters']['Doppler'][k],4),
        v['reactor_parameters']['w_keff'][k],v['reactor_parameters']['w_void'][k],v['reactor_parameters']['w_dopp'][k],v['reactor_parameters']['w_pu'][k],))
    

In [ ]:
temp_des = []
temp_obj = []
lvl_3 = bb.get_attr('lvl_3')
for k,v in lvl_3.items():
    temp_des.append((v['reactor_parameters']['w_keff'][k],v['reactor_parameters']['w_void'][k],v['reactor_parameters']['w_dopp'][k],v['reactor_parameters']['w_pu'][k]))
    temp_obj.append((v['reactor_parameters']['keff'][k],v['reactor_parameters']['void'][k],v['reactor_parameters']['Doppler'][k],v['reactor_parameters']['pu_content'][k]))
print(len(temp_des))
sm = tm.Surrogate_Models()
sm.random=0
sm.clear_surrogate_model()
sm.update_database(temp_obj, temp_des)
model = 'lr'
sm.update_model(model)
sm.optimize_model(model)
sm.predict(model, [[1.0303, -110.023, -0.6926, 0.5475]])
print(sm.models[model]['mse_score'], sm.models[model]['score'])